In [ ]:
!pip install inference-sdk

In [ ]:
!pip install pytesseract

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 32 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (25.3 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121749 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [ ]:
!cp -r /content/drive/MyDrive/CourtVision/* /content

In [ ]:
!pip install ultralytics

In [ ]:
import os
import random
import numpy as np
import cv2
from ultralytics import YOLO
from tracker import Tracker
from scipy.spatial import ConvexHull
from sklearn.cluster import KMeans
import json
from tqdm import tqdm
import gc
from inference_sdk import InferenceHTTPClient
import pytesseract

In [ ]:
pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

In [ ]:
def identify_text(frame, x, y, width, height):
  text = pytesseract.image_to_string(frame[int(y):int(y + height), int(x):int(x + width)])
  return text

In [ ]:
def generate_tracking_data(filename, video_path):
    cap = cv2.VideoCapture(video_path + filename)
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    ret, frame = cap.read()

    player_ids = {}
    player_ids["HOME"] = [None] * 5
    player_ids["AWAY"] = [None] * 5

    model = YOLO("yolov8n.pt")
    CLIENT = InferenceHTTPClient(
      api_url="https://detect.roboflow.com",
      api_key="b97QcNAHuMlKCs9ja7Cl"
    )

    tracker = Tracker()

    colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(13)]

    json_out = {}
    json_out["GAME_PARAMS"] = {}

    frame_num = 0
    while ret:
        frame_num += 1
        print("Frame " + str(frame_num) + "/" + str(num_frames))
        results = model(frame)

        cv2.imwrite('frame_img.jpg', frame)
        game_params = CLIENT.infer("frame_img.jpg", model_id="basketball-players-fy4c2/20")

        predictions = game_params["predictions"]
        prediction_text = []

        for i in predictions:
          temp = {}
          temp["CLASS"] = i["class"]
          temp["CONFIDENCE"] = i["confidence"]
          temp["TEXT"] = identify_text(frame, i["x"], i["y"], i["width"], i["height"])
          temp["FRAME_NUM"] = frame_num
          prediction_text.append(temp)


        json_out["GAME_PARAMS"]["FRAME_NUM"] = prediction_text

        for person in results:
            detections = []
            for r in person.boxes.data.tolist():
                x1, y1, x2, y2, confidence, class_type = r
                x1 = int(x1)
                x2 = int(x2)
                y1 = int(y1)
                y2 = int(y2)

                class_type = int(class_type)
                detections.append([x1, y1, x2, y2, confidence])

            tracker.update(frame, detections)

            for track in tracker.tracks:

                bbox = track.bbox
                track_id = track.track_id
                x1, y1, x2, y2 = bbox

                #Extract jersey num
                try:
                  player_box_img = frame[int(y1):int(y2), int(x1):int(x2)]
                  gray_frame = cv2.cvtColor(player_box_img, cv2.COLOR_BGR2GRAY)

                  cv2.imwrite('temp_frame.png', gray_frame)

                  text = pytesseract.image_to_string('temp_frame.png', lang="eng", config='--psm 6')
                except:
                  text = None

                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]))
                cv2.putText(frame, str(track_id),(int(x1), int(y1)),0, 5e-3 * 200, (colors[track_id % len(colors)]),2)

                if track_id not in json_out.keys():
                    json_out[track_id] = {
                        "FRAME_NUM": [],
                        "X1": [],
                        "Y1": [],
                        "X2": [],
                        "Y2": [],
                        "REC_NUM": []
                    }

                json_out[track_id]["FRAME_NUM"].append(frame_num)
                json_out[track_id]["X1"].append(int(x1))
                json_out[track_id]["Y1"].append(int(y1))
                json_out[track_id]["X2"].append(int(x2))
                json_out[track_id]["Y2"].append(int(y2))
                json_out[track_id]["REC_NUM"].append(text)

        #cv2.imshow('frame', frame)
        #cv2.waitKey(5)
        ret, frame = cap.read()

    cap.release()
    cv2.destroyAllWindows()

    json_out["META"] = {"NUM_FRAMES_IN_VIDEO": num_frames}

    return json_out

In [ ]:
filename = "Footage/Cavs/Offense/test1.mp4"
video_path = ""
video_out_path = ""

json_out = generate_tracking_data(filename, video_path)
json_file_path = "test_out.json"

with open(json_file_path, 'w') as json_file:
    json.dump(json_out, json_file)

In [ ]:
json_out["GAME_PARAMS"]

{'FRAME_NUM': {'time': 0.042576564000228245,
  'image': {'width': 1280, 'height': 720},
  'predictions': [{'x': 203.75,
    'y': 673.75,
    'width': 57.5,
    'height': 35.0,
    'confidence': 0.8838939666748047,
    'class': 'Team Points',
    'class_id': 3,
    'detection_id': '95477efe-4368-4e68-9bc9-6d727208d30d'},
   {'x': 422.5,
    'y': 673.75,
    'width': 67.5,
    'height': 35.0,
    'confidence': 0.8465325832366943,
    'class': 'Team Points',
    'class_id': 3,
    'detection_id': 'bc44d2d1-c0dc-4469-b1f9-bb3011127c1b'},
   {'x': 548.75,
    'y': 673.75,
    'width': 62.5,
    'height': 30.0,
    'confidence': 0.8417928218841553,
    'class': 'Time Remaining',
    'class_id': 4,
    'detection_id': '42ec1cb5-4045-4b88-976e-9d9cfab675e0'},
   {'x': 484.375,
    'y': 675.0,
    'width': 41.25,
    'height': 27.5,
    'confidence': 0.812049150466919,
    'class': 'Period',
    'class_id': 0,
    'detection_id': '17762453-32e3-483a-a84a-aeb3099b7e7e'},
   {'x': 611.875,
    'y

In [ ]:
for filename in tqdm(os.listdir("drive/MyDrive/CourtVision/Footage/Cavs/Offense")):
    if filename.endswith(".mp4"):
      json_out = generate_tracking_data(filename, "drive/MyDrive/CourtVision/Footage/Cavs/Offense/")

      with open("drive/MyDrive/CourtVision/OUT/OUT_" + filename + ".json", 'w') as json_file:
        json.dump(json_out, json_file)

      del json_out
      gc.collect()

In [ ]:
json_out["GAME_PARAMS"]

{'FRAME_NUM': {'time': 0.042576564000228245,
  'image': {'width': 1280, 'height': 720},
  'predictions': [{'x': 203.75,
    'y': 673.75,
    'width': 57.5,
    'height': 35.0,
    'confidence': 0.8838939666748047,
    'class': 'Team Points',
    'class_id': 3,
    'detection_id': '95477efe-4368-4e68-9bc9-6d727208d30d'},
   {'x': 422.5,
    'y': 673.75,
    'width': 67.5,
    'height': 35.0,
    'confidence': 0.8465325832366943,
    'class': 'Team Points',
    'class_id': 3,
    'detection_id': 'bc44d2d1-c0dc-4469-b1f9-bb3011127c1b'},
   {'x': 548.75,
    'y': 673.75,
    'width': 62.5,
    'height': 30.0,
    'confidence': 0.8417928218841553,
    'class': 'Time Remaining',
    'class_id': 4,
    'detection_id': '42ec1cb5-4045-4b88-976e-9d9cfab675e0'},
   {'x': 484.375,
    'y': 675.0,
    'width': 41.25,
    'height': 27.5,
    'confidence': 0.812049150466919,
    'class': 'Period',
    'class_id': 0,
    'detection_id': '17762453-32e3-483a-a84a-aeb3099b7e7e'},
   {'x': 611.875,
    'y